In [9]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(dirname(pwd()) * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

┌ Warning: Replacing docs for `Main.WaterRegulation.connect_reservoirs :: Tuple{Reservoir, Array{Reservoir}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.find_us_reservoir :: Tuple{Reservoir}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.OtherParticipant :: Tuple{Array{Participant}, Participant, Array{Reservoir}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.calculate_balance :: Tuple{Dict{Reservoir, Float64}, Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Reservoir}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243


┌ Warning: Replacing docs for `Main.WaterRegulation.adjust_flow! :: Union{Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{HydropowerPlant, Float64}}, Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{HydropowerPlant, Float64}, Any}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.update_reservoir :: Tuple{Dict{Reservoir, Float64}, Int64}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.update_ind_reservoir :: Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{Reservoir, Float64}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.read_data :: Tuple{String}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs f

┌ Warning: Replacing docs for `Main.WaterRegulation.RealTimeBalancing :: Tuple{Vector{Reservoir}, Participant, Dict{Reservoir, Float64}, Dict{Reservoir, Float64}, Dict{Int64, Float64}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243


### Simulations of Water Regulation Procedure

We would like to set up a skeleton for the day-to-day decision making of hydropower producers and the Water Regulation Company. The main steps of the Water Regulation company consist of:

* Determining a Reference Flow for the next nomination period (Inflow Forecasting)
* Receiving pre-bid nominations and calculating adjusted flow, power swaps, etc. -> Send these parameters back to the participants
* Recalculate based on renominations by power producers after bidding

For the Power Producers / Participants:  

* Fetch and Calculate all Parameters for the Water Regulation Procedure (Individual Reservoir Level etc.)
* Price and Inflow Forecasting (Fix Price Points)
* Medium-Term Hydropower Model Calculation, obtain Water Value Cuts
* Decision Model 1: Bidding and Pre-Market Clearing Nomination
* Decision Model 2: Short-Term Optimization and Renomination
* Decision Model 3: Fixed Water Flow scheduling and real time balancing

This constitutes one round of simulation.


In [10]:
filepath_Ljungan = dirname(pwd()) * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = dirname(pwd()) *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = dirname(pwd()) * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)

println("The reservoir system consists of $([r.dischargepoint for r in R])")
println("Downstream of the reservoirs we have power plants $([k.name for k in K])")
for j in J
    println("$(j.name) owns $(j.plants), which constitutes a participation rate of: \n ", j.participationrate)
end


The reservoir system consists of ["Flasjon", "Holsmjon"]
Downstream of the reservoirs we have power plants ["Flasjo", "Trangfors", "Ratan", "Turinge", "Bursnas", "Jarnvagsforsen", "Parteboda", "Hermansboda", "Ljunga", "Nederede", "Skallbole", "Matfors", "Viforsen"]
Sydkraft owns HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas], which constitutes a participation rate of: 
 

________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Fortum owns HydropowerPlant[Parteboda, Hermansboda, Ljunga], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 0.8200000000000001
Holsmjon | 0.8200000000000001

Statkraft owns HydropowerPlant[Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 1.86    
Holsmjon | 1.86    



#### Load Data

Other than the river system, the relevant data for optimization is organized in Dataframes.
We load in cleaned versions of the data.

In [11]:
filepath_prices = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Spot Prices\\prices_df.csv"
filepath_inflows = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Inflow\\Data from Flasjoen and Holmsjoen.csv"

inflow_data = prepare_inflowdata(filepath_inflows)
price_data = prepare_pricedata(filepath_prices)

Row,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Average,season,Weekday,CalendarWeek
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String,Int64
1,2020-12-15,17.57,17.57,16.92,16.88,18.95,20.39,22.31,24.59,31.41,31.49,31.62,30.52,35.9,38.1,40.08,43.08,37.32,51.13,47.54,40.77,35.2,28.45,23.67,21.86,30.1383,Winter,Weekday,51
2,2020-12-16,20.48,19.32,19.0,19.02,20.19,21.98,22.71,32.71,48.37,43.68,42.01,37.92,36.56,36.42,39.16,46.54,46.4,39.41,35.78,24.74,22.78,22.15,21.48,20.12,30.7887,Winter,Weekday,51
3,2020-12-17,19.03,17.93,17.53,17.28,17.97,20.0,22.09,23.64,26.79,25.28,24.43,24.3,23.67,23.99,23.97,25.06,25.38,26.07,24.22,23.16,22.19,21.97,20.76,18.99,22.3208,Winter,Weekday,51
4,2020-12-18,16.96,16.65,16.17,16.3,17.3,19.39,19.99,22.43,23.74,23.96,22.82,22.5,22.44,22.23,22.42,22.91,23.11,23.83,22.59,21.01,19.43,19.04,17.82,15.92,20.4567,Winter,Weekday,51
5,2020-12-19,16.48,15.98,15.47,15.31,15.3,15.52,15.87,15.92,16.48,17.01,17.36,17.47,17.27,17.29,17.33,17.54,17.96,18.39,17.77,17.03,15.92,15.69,15.22,13.65,16.4679,Winter,Weekend,51
6,2020-12-20,13.0,12.04,11.79,11.65,11.79,11.98,12.53,12.7,13.25,14.28,15.37,15.94,15.83,15.37,15.98,16.84,17.09,17.29,17.06,16.3,15.32,15.23,14.65,13.02,14.4292,Winter,Weekend,51
7,2020-12-21,11.9,11.2,11.5,11.73,12.81,14.37,16.67,19.74,20.6,19.92,19.32,19.39,19.44,19.2,19.45,20.65,20.55,20.35,19.49,18.03,16.17,14.47,13.18,8.66,16.6163,Winter,Weekday,52
8,2020-12-22,7.15,5.51,4.77,4.44,4.84,7.73,12.82,15.97,17.87,18.0,18.09,18.01,18.01,17.57,18.08,18.62,19.67,19.87,19.79,18.01,16.88,16.05,15.05,13.71,14.4379,Winter,Weekday,52
9,2020-12-23,13.38,13.1,12.9,12.93,13.14,14.72,15.99,18.59,19.21,19.51,18.97,19.33,19.55,19.4,19.66,19.96,20.56,21.79,20.77,18.95,17.82,16.94,16.16,15.07,17.4333,Winter,Weekday,52


### Price Forecasting

Price is sensitive to various factors. Daytime, Weekday and time of year play a relevant role in how prices usually behave.

* For the bidding and short-term optimization problems, part of the uncertainty lies in unknown hourly prices.

* For the medium term problem, we are interested in daily (average) prices.

* We want to obtain Price Points which will be important in the analysis from the historic data.

* We need penalty parameters for using the balancing markets, which could be obtained from using maximum / minimum values fromm observed spot market prices / generated scenarios.

In [24]:
const scenario_count_prices_hourly = 10
const scenario_count_prices_weekly = 2
const stage_count = 8
const price_point_count = 5

PriceScenariosMedium = Price_Scenarios_Weekly(price_data, scenario_count_prices_weekly)
PriceScenariosShort = Price_Scenarios_Hourly(price_data, scenario_count_prices_hourly)
PPoints = Create_Price_Points(price_data, 5)
mu_up, mu_down = BalanceParameters(PriceScenariosShort)

Tuple{Float64, Float64}

### Inflow Forecasting

Inflow has extreme seasonal differences, depending on geographical location.  
For the Ljungan River System we could observe that most inflow occurs during the spring melt around May.

In [5]:
#InflowScenariosShort = inflow_generation_short(inflow_data)
const ColumnReservoir = Dict{Reservoir, String}(R[1] => "Flasjon Inflow", R[2] => "Holmsjon Inflow")
const scenario_count_inflows_weekly = 2
InflowScenariosMedium = Inflow_Scenarios_Weekly(inflow_data, ColumnReservoir, scenario_count_inflows_weekly, R)


UndefVarError: UndefVarError: `Inflow_Scenarios_Weekly` not defined

### Create Uncertainty Set

Depending on the Problem, we create the uncertainty sets from both price and inflow Scenarios.
This becomes more difficult for the Anticipatory Bidding Problem.

In [6]:
Ω_medium, P_medium =  create_Ω_medium(PriceScenariosMedium, InflowScenariosMedium, R)

Ω_NA, P_NA = create_Ω_Nonanticipatory(PriceScenariosShort, )

Ω_A, P_A = create_Ω_Anticipatory()

UndefVarError: UndefVarError: `create_Ω_medium` not defined

### Reference Flow

To calculate a reference flow for each reservoir, the Water Regulation Company has to combine information from inflow, historical reservoir trajectories and current reservoir level.  

In [7]:
function calculate_reference_flow(R::Array{Reservoir}, l_traj::Dict{Reservoir, Float64}, Qinflow::Dict{Reservoir, Float64})::Dict{Reservoir, Float64}
    Qref = Dict{Reservoir, Float64}()
    for r in R
        Qref[r] = (r.currentvolume - l_traj[r]) / T + Qinflow[r]
    end
    return Qref
end

calculate_reference_flow (generic function with 1 method)

## Participants

Every Participant individually plans their bidding and production. Irrespective of the strategy, only limited information is available. Internal to the participants problems are 

* Price Forecasts
* Nomination to Water Regulation Company
* Bidding at Electricity Market

### Medium Term Problem

The Medium Term / Seasonal Optimization Problem is necessary to obtain Cuts for the Water Value Function.

Therefor the point of a functioin containing this decision model is to return cut coefficients of the Water Value Function.

* What are the inputs to the Medium Term problem?
* How are the cuts calculated?
* How many cuts should be calculated?

In [8]:
function MediumTermOptimization()
    return
end

function WaterValueCutGeneration()
    return
end

WaterValueCutGeneration (generic function with 1 method)

### Bidding Problem


### Short Term Optimization

### Real Time Balancing and Scheduling

# Simulation

For a preset amount of rounds, we simulate the reservoir system through the functions and structures as described above. It is important to have a way to save results adequately! This encapsulates the calculations, and evaluations and visualizations can take place at a later point in time.